# EDA Instacart


## 1) Checagem inicial dos dados


In [3]:
# Importacao e carga dos dados
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

BASE_DIR = Path.cwd().parent  # sobe de notebooks para Dashboard_Instacart
df_orders = pd.read_csv(BASE_DIR / "orders.csv", sep=";")
df_products = pd.read_csv(BASE_DIR / "products.csv", sep=";")
df_aisles = pd.read_csv(BASE_DIR / "aisles.csv", sep=";")
df_departments = pd.read_csv(BASE_DIR / "departments.csv", sep=";")
df_order_products = pd.read_csv(BASE_DIR / "order_products.csv", sep=";")


In [4]:
# Visao geral das tabelas
df_orders.info()
df_products.info()
df_aisles.info()
df_departments.info()
df_order_products.info(verbose=True, show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478967 entries, 0 to 478966
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                478967 non-null  int64  
 1   user_id                 478967 non-null  int64  
 2   order_number            478967 non-null  int64  
 3   order_dow               478967 non-null  int64  
 4   order_hour_of_day       478967 non-null  int64  
 5   days_since_prior_order  450148 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 21.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49694 entries, 0 to 49693
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49694 non-null  int64 
 1   product_name   48436 non-null  object
 2   aisle_id       49694 non-null  int64 
 3   department_id  49694 non-null  int64 
dtypes: int64(3), object(1)
memory usage:

In [5]:
# Duplicados e filtros iniciais em orders
print(f"orders duplicated rows: {df_orders.duplicated().sum()}")
print(
    "orders on dow=3 hour=2: ",
    df_orders[(df_orders["order_dow"] == 3)
             & (df_orders["order_hour_of_day"] == 2)].shape[0],
)
print(f"orders duplicated order_id: {df_orders["order_id"].duplicated().sum()}")


orders duplicated rows: 15
orders on dow=3 hour=2:  121
orders duplicated order_id: 15


In [6]:
# Duplicados em products
print(f"products duplicated rows: {df_products.duplicated().sum()}")
print(
    "products duplicated product_id: "
    f"{df_products["product_id"].duplicated().sum()}"
)
print(
    "products product_name nulls: "
    f"{df_products["product_name"].isna().sum()}"
)


products duplicated rows: 0
products duplicated product_id: 0
products product_name nulls: 1258


In [7]:
# Duplicados em departments
print(f"departments duplicated rows: {df_departments.duplicated().sum()}")
print(
    "departments duplicated department_id: "
    f"{df_departments["department_id"].duplicated().sum()}"
)


departments duplicated rows: 0
departments duplicated department_id: 0


In [8]:
# Duplicados em aisles
print(f"aisles duplicated rows: {df_aisles.duplicated().sum()}")
print(
    "aisles duplicated aisle_id: "
    f"{df_aisles["aisle_id"].duplicated().sum()}"
)


aisles duplicated rows: 0
aisles duplicated aisle_id: 0


In [9]:
# Duplicados e nulos em order_products
print(f"order_products duplicated rows: {df_order_products.duplicated().sum()}")
print(
    "order_products add_to_cart_order nulls: "
    f"{df_order_products["add_to_cart_order"].isnull().sum()}"
)


order_products duplicated rows: 0
order_products add_to_cart_order nulls: 836


In [10]:
# Investigando produtos com nome ausente
print((df_products["aisle_id"] == 100).sum())
print((df_products["department_id"] == 21).sum())
print(
    df_products[(df_products["department_id"] == 21) 
    & (df_products["aisle_id"] == 100)]
)


1258
1258
       product_id product_name  aisle_id  department_id
37             38          NaN       100             21
71             72          NaN       100             21
109           110          NaN       100             21
296           297          NaN       100             21
416           417          NaN       100             21
...           ...          ...       ...            ...
49552       49553          NaN       100             21
49574       49575          NaN       100             21
49640       49641          NaN       100             21
49663       49664          NaN       100             21
49668       49669          NaN       100             21

[1258 rows x 4 columns]


In [11]:
# Verificacao cruzada: aisle_id=100 e department_id=21
aisle_100 = df_aisles[df_aisles["aisle_id"] == 100]
department_21 = df_departments[df_departments["department_id"] == 21]
print("Aisle 100:\n", aisle_100)
print("Department 21:\n", department_21)


Aisle 100:
     aisle_id    aisle
99       100  missing
Department 21:
     department_id department
20             21    missing


In [12]:
# Nulos e estatisticas em orders
print(
    "orders days_since_prior_order nulls: ",
    df_orders["days_since_prior_order"].isna().sum(),
)
print(
    "orders order_number==1 with days_since_prior_order null: ",
    df_orders[(df_orders["order_number"] == 1)
             & (df_orders["days_since_prior_order"].isna())].shape[0],
)


orders days_since_prior_order nulls:  28819
orders order_number==1 with days_since_prior_order null:  28819


In [13]:
# Nulos e estatisticas em order_products
print("order_products null counts:\n", df_order_products.isna().sum())
print(
    "order_products order_id min/max:\n",
    df_order_products["order_id"].agg(["min", "max"]),
)
print(
    "order_products product_id min/max:\n",
    df_order_products["product_id"].agg(["min", "max"]),
)
print(
    "order_products add_to_cart_order min/max:\n",
    df_order_products["add_to_cart_order"].agg(["min", "max"]),
)
print(
    "order_products reordered min/max:\n",
    df_order_products["reordered"].agg(["min", "max"]),
)


order_products null counts:
 order_id               0
product_id             0
add_to_cart_order    836
reordered              0
dtype: int64
order_products order_id min/max:
 min          4
max    3421079
Name: order_id, dtype: int64
order_products product_id min/max:
 min        1
max    49694
Name: product_id, dtype: int64
order_products add_to_cart_order min/max:
 min     1.0
max    64.0
Name: add_to_cart_order, dtype: float64
order_products reordered min/max:
 min    0
max    1
Name: reordered, dtype: int64


In [14]:
# Orders com add_to_cart_order ausente
nan_numbers_id = df_order_products.loc[
    df_order_products["add_to_cart_order"].isna(), "order_id"
].nunique()
print(f"order_products orders with add_to_cart_order null: {nan_numbers_id}")


order_products orders with add_to_cart_order null: 70


In [15]:
# Todos os pedidos com valores ausentes contêm mais de 64 produtos?
min_items_missing = (
    df_order_products[df_order_products["add_to_cart_order"].isna()]
    .groupby("order_id")["product_id"]
    .count()
    .min()
)
print(f"missing add_to_cart_order min items per order: {min_items_missing}")
print(f"missing add_to_cart_order min items > 64: {min_items_missing > 64}")


missing add_to_cart_order min items per order: 1
missing add_to_cart_order min items > 64: False


In [16]:
# Agrupando os pedidos com dados ausentes por ID de pedido
print(
    "missing add_to_cart_order orders: ",
    df_order_products[df_order_products["add_to_cart_order"].isna()]
    .groupby("order_id")
    .size()
    .shape[0],
)


missing add_to_cart_order orders:  70


## 2) Padronizacao e limpeza


In [17]:
# Remocao de duplicados em orders
df_orders = df_orders.drop_duplicates()
print(f"orders shape after drop_duplicates: {df_orders.shape}")
print(f"orders duplicated rows after drop: {df_orders.duplicated().sum()}")
print(f"orders duplicated order_id: {df_orders["order_id"].duplicated().sum()}")


orders shape after drop_duplicates: (478952, 6)
orders duplicated rows after drop: 0
orders duplicated order_id: 0


In [18]:
# Padronizando nomes de produtos
df_products["product_name"] = df_products["product_name"].str.lower()
print(
    "products duplicated product_name (non-null): ",
    df_products.dropna(subset=["product_name"])["product_name"]
    .duplicated()
    .sum(),
)


products duplicated product_name (non-null):  104


In [19]:
# Preenchendo nomes de produtos ausentes
df_products = df_products.fillna("Unknown")


In [20]:
# Ajuste de tipos em orders
df_orders["days_since_prior_order"] = (
    df_orders["days_since_prior_order"].fillna(0).astype(int)
)


In [21]:
# Substituindo valores ausentes em add_to_cart_order
df_order_products["add_to_cart_order"] = (
    df_order_products["add_to_cart_order"].fillna(999).astype(int)
)


**Vizualizando os DataFrames novamente**

In [22]:
df_orders.info()
df_products.info()
df_aisles.info()
df_departments.info()
df_order_products.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 478952 entries, 0 to 478966
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   order_id                478952 non-null  int64
 1   user_id                 478952 non-null  int64
 2   order_number            478952 non-null  int64
 3   order_dow               478952 non-null  int64
 4   order_hour_of_day       478952 non-null  int64
 5   days_since_prior_order  478952 non-null  int64
dtypes: int64(6)
memory usage: 25.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49694 entries, 0 to 49693
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49694 non-null  int64 
 1   product_name   49694 non-null  object
 2   aisle_id       49694 non-null  int64 
 3   department_id  49694 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB
<class 'pandas.core.fram

## 3) Analise e insights


In [23]:
# Mapeamentos de dias e horas
day_names = {
    0: "Domingo",
    1: "Segunda",
    2: "Terça",
    3: "Quarta",
    4: "Quinta",
    5: "Sexta",
    6: "Sábado",
}
hour_labels = [
    f"{h:2d}am" if h < 12 else ("12pm" if h == 12 else f"{h-12:2d}pm")
    for h in range(24)
]


In [24]:
# Quantas pessoas fazem pedidos a cada hora do dia?
people_per_hour = (
    df_orders.groupby("order_hour_of_day")["user_id"]
    .nunique()
    .reindex(range(24), fill_value=0)
)
people_per_hour_df = people_per_hour.reset_index()
people_per_hour_df.columns = ["order_hour_of_day", "people_count"]
people_per_hour_df["hour_label"] = people_per_hour_df["order_hour_of_day"].map(
    dict(enumerate(hour_labels))
)
fig = px.bar(
    people_per_hour_df,
    x="hour_label",
    y="people_count",
    title="Pessoas fazendo pedidos a cada hora do dia",
    labels={
        "hour_label": "Hora do dia",
        "people_count": "Quantidade de pessoas",
    },
    color_discrete_sequence=["hotpink"],
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [25]:
# Em que dia da semana as pessoas compram produtos alimenticios?
day_of_week_orders = (
    df_orders.groupby("order_dow")["user_id"]
    .nunique()
    .reindex(range(7), fill_value=0)
)
day_of_week_df = day_of_week_orders.reset_index()
day_of_week_df.columns = ["order_dow", "people_count"]
day_of_week_df["day_name"] = day_of_week_df["order_dow"].map(day_names)
fig = px.bar(
    day_of_week_df,
    x="day_name",
    y="people_count",
    title="Dia da semana que as pessoas mais fazem compras",
    labels={
        "day_name": "Dias da semana",
        "people_count": "Quantidade de pessoas",
    },
    color_discrete_sequence=["hotpink"],
    category_orders={"day_name": [day_names[i] for i in range(7)]},
)
fig.show()


In [26]:
# Quanto tempo as pessoas esperam ate fazer outro pedido?
days_until_next_order = (
    df_orders.groupby("days_since_prior_order")["user_id"]
    .nunique()
    .sort_index()
)
days_until_df = days_until_next_order.reset_index()
days_until_df.columns = ["days_since_prior_order", "people_count"]
days_until_df["days_label"] = days_until_df["days_since_prior_order"].astype(int).astype(str)
fig = px.bar(
    days_until_df,
    x="days_label",
    y="people_count",
    title="Quanto tempo as pessoas levam ate seu proximo pedido",
    labels={
        "days_label": "Dias ate o proximo pedido",
        "people_count": "Quantidade de pessoas",
    },
    color_discrete_sequence=["hotpink"],
)
fig.show()


In [27]:
# Heatmap: pedidos por dia da semana e hora
heatmap_df = (
    df_orders.groupby(["order_dow", "order_hour_of_day"])["order_id"]
    .count()
    .reset_index(name="order_count")
)
heatmap_df["day_name"] = heatmap_df["order_dow"].map(day_names)
heatmap_df["hour_label"] = heatmap_df["order_hour_of_day"].map(
    dict(enumerate(hour_labels))
)
fig = px.density_heatmap(
    heatmap_df,
    x="hour_label",
    y="day_name",
    z="order_count",
    color_continuous_scale="Reds",
    title="Heatmap de pedidos por dia e hora",
    category_orders={"day_name": [day_names[i] for i in range(7)]},
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [28]:
# Pedidos por hora com botoes por dia da semana
orders_by_hour_all = (
    df_orders.groupby(["order_dow", "order_hour_of_day"])["user_id"]
    .count()
    .reset_index(name="order_count")
)
orders_by_hour_all["hour_label"] = orders_by_hour_all["order_hour_of_day"].map(
    dict(enumerate(hour_labels))
)
fig = go.Figure()
for dow in range(7):
    data = orders_by_hour_all[orders_by_hour_all["order_dow"] == dow]
    fig.add_bar(
        x=data["hour_label"],
        y=data["order_count"],
        name=day_names[dow],
        visible=(dow == 0),
    )

buttons = []
for i in range(7):
    visibility = [False] * 7
    visibility[i] = True
    buttons.append(
        dict(
            label=day_names[i],
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Pedidos por hora - {day_names[i]}"},
            ],
        )
    )

fig.update_layout(
    title=f"Pedidos por hora - {day_names[0]}",
    xaxis_title="Horas do dia",
    yaxis_title="Quantidade de pedidos",
    xaxis_tickangle=90,
    updatemenus=[dict(active=0, buttons=buttons, x=1.02, y=1)],
)
fig.show()


In [29]:
# Dispersao com slider por dia da semana
orders_by_hour_day = (
    df_orders.groupby(["order_dow", "order_hour_of_day"])["order_id"]
    .count()
    .reset_index(name="order_count")
)
orders_by_hour_day["day_name"] = orders_by_hour_day["order_dow"].map(day_names)
orders_by_hour_day["hour_label"] = orders_by_hour_day["order_hour_of_day"].map(
    dict(enumerate(hour_labels))
)
fig = px.scatter(
    orders_by_hour_day,
    x="hour_label",
    y="order_count",
    animation_frame="day_name",
    title="Pedidos por hora (slider por dia da semana)",
    labels={
        "hour_label": "Hora do dia",
        "order_count": "Numero de pedidos",
        "day_name": "Dia da semana",
    },
    category_orders={"day_name": [day_names[i] for i in range(7)]},
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [30]:
# Distribuicao do numero de pedidos por cliente
order_per_client = (
    df_orders.groupby("user_id")["order_id"]
    .count()
    .value_counts()
    .sort_index()
)
order_per_client_df = order_per_client.reset_index()
order_per_client_df.columns = ["orders_count", "clients_count"]
fig = px.bar(
    order_per_client_df,
    x="orders_count",
    y="clients_count",
    title="Numero de pedidos por cliente",
    labels={
        "orders_count": "Numero de pedidos",
        "clients_count": "Quantidade de clientes",
    },
    log_y=True,
)
fig.show()


In [31]:
# 20 produtos mais populares
merged_dfs = df_order_products.merge(
    df_products[["product_id", "product_name"]], on="product_id"
)
most_popular_itens_general = (
    merged_dfs.groupby(["product_id", "product_name"])["order_id"]
    .count()
    .sort_values(ascending=False)
    .head(20)
)
most_popular_itens_general_df = most_popular_itens_general.reset_index()
most_popular_itens_general_df.columns = [
    "product_id",
    "product_name",
    "order_count",
]
most_popular_itens_general_df["product_label"] = (
    most_popular_itens_general_df["product_name"]
    + " ("
    + most_popular_itens_general_df["product_id"].astype(str)
    + ")"
)
fig = px.bar(
    most_popular_itens_general_df,
    x="product_label",
    y="order_count",
    title="20 Pedidos mais populares",
    labels={
        "product_label": "Nome e ID dos produtos",
        "order_count": "Quantidade de pedidos",
    },
    color_discrete_sequence=["gold"],
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [32]:
# Distribuicao de itens por pedido (ate 99%)
count_products_ordered = df_order_products.groupby("order_id")["product_id"].count()
max_x = int(count_products_ordered.quantile(0.99))
count_products_df = (
    count_products_ordered[count_products_ordered <= max_x]
    .to_frame(name="items_per_order")
    .reset_index(drop=True)
)
fig = px.histogram(
    count_products_df,
    x="items_per_order",
    nbins=max_x,
    title="Distribuicao de itens por pedido (ate 99%)",
    labels={
        "items_per_order": "Itens por pedido",
        "count": "Numero de pedidos",
    },
    color_discrete_sequence=["skyblue"],
)
fig.update_layout(xaxis_range=[0.5, max_x + 0.5])
fig.show()


In [33]:
# 20 principais itens em pedidos repetidos
merged_df = df_order_products.merge(
    df_products[["product_id", "product_name"]], on="product_id"
)
most_popular_itens = (
    merged_df[merged_df["reordered"] == 1]
    .groupby(["product_id", "product_name"])["order_id"]
    .count()
    .sort_values(ascending=False)
    .head(20)
)
most_popular_itens_df = most_popular_itens.reset_index()
most_popular_itens_df.columns = ["product_id", "product_name", "order_count"]
most_popular_itens_df["product_label"] = (
    most_popular_itens_df["product_name"]
    + " ("
    + most_popular_itens_df["product_id"].astype(str)
    + ")"
)
fig = px.bar(
    most_popular_itens_df,
    x="product_label",
    y="order_count",
    title="20 Produtos mais populares em pedidos repetidos",
    labels={
        "product_label": "Nome e ID dos produtos",
        "order_count": "Quantidade de pedidos",
    },
    color_discrete_sequence=["gold"],
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [34]:
# 20 itens mais colocados primeiro no carrinho
first_items = merged_dfs[merged_dfs["add_to_cart_order"] == 1]
top20_first = (
    first_items.groupby(["product_id", "product_name"])
    .size()
    .sort_values(ascending=False)
    .head(20)
)
top20_first_df = top20_first.reset_index()
top20_first_df.columns = ["product_id", "product_name", "first_count"]
top20_first_df["product_label"] = (
    top20_first_df["product_name"]
    + " ("
    + top20_first_df["product_id"].astype(str)
    + ")"
)
fig = px.bar(
    top20_first_df,
    x="product_label",
    y="first_count",
    title="20 Principais produtos colocados no carrinho primeiro",
    labels={
        "product_label": "Nome e ID dos produtos",
        "first_count": "Numero de vezes que foi o primeiro item",
    },
    color_discrete_sequence=["khaki"],
)
fig.update_layout(xaxis_tickangle=90)
fig.show()


In [35]:
# Proporcao de itens repetidos por cliente
df_op = df_order_products.merge(df_orders[["order_id", "user_id"]], on="order_id")
proporcao_pedidos_repetidos_cliente = df_op.groupby("user_id")["reordered"].mean()
tabela_proporcao_repetidos_cliente = pd.DataFrame(
    {
        "user_id": proporcao_pedidos_repetidos_cliente.index,
        "proporcao_repetidos": proporcao_pedidos_repetidos_cliente.values,
    }
)
proporcao_pct = proporcao_pedidos_repetidos_cliente * 100
proporcao_df = proporcao_pct.reset_index()
proporcao_df.columns = ["user_id", "proporcao_pct"]
mean_prop = proporcao_pct.mean()
median_prop = proporcao_pct.median()
fig = px.histogram(
    proporcao_df,
    x="proporcao_pct",
    nbins=50,
    title="Distribuicao da proporcao de itens repetidos por cliente",
    labels={
        "proporcao_pct": "Proporcao de itens repetidos (%)",
        "count": "Quantidade de clientes",
    },
    color_discrete_sequence=["teal"],
)
fig.add_vline(
    x=mean_prop, line_dash="dash", line_color="orange", annotation_text="Media"
)
fig.add_vline(
    x=median_prop, line_dash="dash", line_color="purple", annotation_text="Mediana"
)
fig.show()
tabela_proporcao_repetidos_cliente.head()


,user_id,proporcao_repetidos
0,2,0.038462
1,4,0.000000
2,5,0.666667
3,6,0.000000
4,7,0.928571


In [36]:
# Dispersao: frequencia do top 5 produtos por days_since_prior_order
top5_products = (
    df_order_products.merge(
        df_products[["product_id", "product_name"]], on="product_id"
    )
    .groupby(["product_id", "product_name"])
    .size()
    .sort_values(ascending=False)
    .head(5)
    .reset_index()[["product_id", "product_name"]]
)
orders_with_days = df_orders[["order_id", "days_since_prior_order"]].copy()
top5_orders = (
    df_order_products.merge(top5_products, on="product_id")
    .merge(orders_with_days, on="order_id")
)
top5_orders = top5_orders.dropna(subset=["days_since_prior_order"])
top5_orders["days_since_prior_order"] = top5_orders["days_since_prior_order"].astype(int)
freq_by_day = (
    top5_orders.groupby(["product_name", "days_since_prior_order"])
    .size()
    .reset_index(name="order_count")
)
fig = px.scatter(
    freq_by_day,
    x="days_since_prior_order",
    y="order_count",
    color="product_name",
    title="Top 5 produtos: frequencia por days_since_prior_order",
    labels={
        "days_since_prior_order": "Dias desde o ultimo pedido",
        "order_count": "Quantidade de pedidos",
        "product_name": "Produto",
    },
    opacity=0.6,
    trendline="ols",
)
fig.show()
